<a href="https://colab.research.google.com/github/fabrizio25504/Enlazados_proyecto_qu-mica/blob/main/Logica_Proposicional_Expresiones_regulares(sin%20terminar).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traducción de frases inglesas a enunciados de lógica proposicional

En un curso de Lógica, un ejercicio consiste en convertir una frase  como ésta:

> *Sieglinde sobrevivirá, y o bien su hijo obtendrá el Anillo y el plan de Wotan se cumplirá o bien el Valhalla será destruido.*

en un enunciado formal de lógica proposicional:

    P ⋀ ((Q ⋀ R) ∨ S)
    
junto con las definiciones de las proposiciones:

    P: Sieglinde sobrevivirá
    P: El hijo de Sieglinde obtendrá el Anillo
    R: El plan de Wotan se cumplirá
    S: El Valhalla será destruido

En el caso de algunas frases, se necesitan conocimientos detallados para conseguir una buena traducción. Las dos frases siguientes son ambiguas, con diferentes interpretaciones preferidas, y traducirlas correctamente requiere conocimientos sobre hábitos alimenticios:

    Comeré ensalada o comeré pan y comeré mantequilla.     P ∨ (Q ⋀ R)
    Comeré ensalada o comeré sopa y comeré helado.        (P ∨ Q) ⋀ R

Pero para muchas frases, el proceso de traducción es automático, sin necesidad de conocimientos especiales.  Desarrollar un programa es importante para manejar estas frases fáciles. El programa se basa en la idea de una serie de reglas de traducción de la forma:

    Regla('{P} ⇒ {Q}', 'si {P} entonces {Q}', 'si {P}, {Q}')
    
lo que significa que la traducción lógica tendrá la forma `'P ⇒ Q'`, siempre que la frase tenga la forma `'si P entonces Q'` o  `'si P, Q'`, donde `P` y `Q` puede coincidir con cualquier sucesión no vacía de caracteres.  Lo que coincida con `P` y `Q` serán procesados recursivamente por las reglas. Las reglas están en ordenserán procesados recursivamente por las reglas. Las reglas están en orden&mdash;de arriba abajo, de izquierda a derecha, y la primera regla que coincida en ese orden será aceptada, sin importar qué, así que asegúrate de ordenar tus reglas cuidadosamente. Una pauta que he seguido es poner todas las reglas que empiezan con una palabra clave (como `'si'` o `'tampoco'`.) antes de las reglas que empiezan por una variable (como `'{P}'`); Así evitará que una palabra clave quede engullida accidentalmente dentro de un `'{P}'`.

Considere la frase de ejemplo `"Si amarte está mal, no quiero tener razón."` Esto debería coincidir con el patrón `'si {P}, {Q}'` with the variable `P` equal to `"amarte está mal"`. Pero no quiero que la variable `Q` sea
`"No quiero tener razón"`, más bien, quiero tener `～Q` igual a `"Quiero tener razón"`. Así que además de tener un conjunto de `Reglas` para manejar el `'si {P}, {Q}'` también tendré una lista de "negaciones" para manejar "no" y similares.

Aquí está el código para procesar las definiciones `Reglas` (usando [expresiones regulares](https://docs.python.org/3.5/library/re.html), lo que a veces puede resultar confuso).

In [1]:
import re

def Rule(output, *patterns):
    "Una regla que produce `salida` si toda la entrada coincide con alguno de los `patrones`."
    return (output, [name_group(pat) + '$' for pat in patterns])

def name_group(pat):
    "Sustituir '{Q}' con '(?P<Q>.+?)', lo que significa 'coincide con 1 o más caracteres, y lo llama Q'"
    return re.sub('{(.)}', r'(?P<\1>.+?)', pat)

def word(w):
    "Devuelve una expresión regular que coincide con w como una palabra completa (no letras dentro de una palabra)."
    return r'\b' + w + r'\b' # '\b' coincide con el límite de la palabra

Veamos cómo es una regla:

In [2]:
Rule('{P} ⇒ {Q}', 'if {P} then {Q}', 'if {P}, {Q}'),


(('{P} ⇒ {Q}',
  ['if (?P<P>.+?) then (?P<Q>.+?)$', 'if (?P<P>.+?), (?P<Q>.+?)$']),)

Y ahora las reglas propiamente dichas. Si su frase no se traduce correctamente, puede intentar aumentar estas reglas para manejar su frase.

In [3]:
rules = [
    Rule('{P} ⇒ {Q}',         'if {P} then {Q}', 'if {P}, {Q}'),
    Rule('{P} ⋁ {Q}',          'either {P} or else {Q}', 'either {P} or {Q}','o {P} o {Q}'),
    Rule('{P} ⋀ {Q}',          'both {P} and {Q}'),
    Rule('～{P} ⋀ ～{Q}',       'neither {P} nor {Q}'),
    Rule('～{A}{P} ⋀ ～{A}{Q}', '{A} neither {P} nor {Q}'), # El Kaiser ni ...
    Rule('～{Q} ⇒ {P}',        '{P} unless {Q}'),
    Rule('{P} ⇒ {Q}',          '{Q} provided that {P}', '{Q} whenever {P}',
                               '{P} implies {Q}', '{P} therefore {Q}',
                               '{Q}, if {P}', '{Q} if {P}', '{P} only if {Q}'),
    Rule('{P} ⋀ {Q}',          '{P} and {Q}', '{P} but {Q}'),
    Rule('{P} ⋁ {Q}',          '{P} or else {Q}', '{P} or {Q}'),
    Rule('{P} ⇒ {Q}', '{Q} debido a que {P}','Si {P} entonces {Q}'),

    Rule('{P} ⇒ {Q}',         'si {P} entonces {Q}', 'si {P}, {Q}', 'si {P} implica {Q}', 'si {P}, {Q}'),
    Rule('{P} ⋁ {Q}',          'o {P} o {Q}', 'ya sea {P} o {Q}', 'o bien {P} o {Q}'),
    Rule('{P} ⋀ {Q}',          'tanto {P} como {Q}', 'ambos {P} y {Q}'),
    Rule('～{P} ⋀ ～{Q}',       'ni {P} ni {Q}', 'no {P} y no {Q}'),
    Rule('～{A}{P} ⋀ ～{A}{Q}', '{A} ni {P} ni {Q}'),
    Rule('～{Q} ⇒ {P}',        '{P} a menos que {Q}', '{P} salvo que {Q}'),
    Rule('{P} ⇒ {Q}',          '{Q} siempre que {P}', '{Q} siempre que {P}', '{P} implica {Q}', '{P} por lo tanto {Q}',
                               '{Q} si {P}', '{Q} si {P}', '{P} solo si {Q}'),
    Rule('{P} ⋀ {Q}',          '{P} y {Q}', '{P} pero {Q}'),
    Rule('{P} ⋁ {Q}',          '{P} o {Q}', '{P} o {Q}'),
    Rule('{P} ⇒ {Q}',          '{Q} debido a {P}', 'si {P}, entonces {Q}'),
    ]

negations = [
    (word("not"), ""),
    (word("cannot"), "can"),
    (word("can't"), "can"),
    (word("won't"), "will"),
    (word("ain't"), "is"),
    (word("no"),"si"),
    ("n't", ""), # coincidencias como parte de una palabra: didn't, couldn't, etc.
    (word("no"), ""),
    (word("nunca"), ""),
    (word("jamás"), ""),
    (word("nada"), ""),
    (word("ningún"), ""),
    (word("ninguna"), ""),
    (word("nunca"), ""),
    (word("siempre"), ""),
    (word("no puede"), ""),
    (word("no se puede"), ""),
    (word("no es posible"), ""),

    ]

Ahora el mecanismo para procesar estas reglas. La función clave es `match_rule`, que compara una frase en inglés con una regla. La función devuelve dos valores, una cadena que representa la traducción de la frase inglesa a la lógica, y `defs`, un diccionario de `{Variable: `value`}` pares. Si `match_rule` encuentra que la regla coincide, llama recursivamente a `match_rules` para que coincida con cada uno de los subgrupos de la expresión regular (la `P` y `Q` en `si {P}, entonces {Q}`).
La función `match_literal` se encarga de las negaciones, y es donde el `defs` el diccionario se actualiza.

In [4]:
def match_rules(sentence, rules, defs):
    """Compara la frase con todas las reglas, aceptando la primera coincidencia; o bien la convierte en un átomo.
    Devuelve dos valores: la traducción lógica y un dict de definiciones {P: 'english'}."""
    sentence = clean(sentence)
    for rule in rules:
        result = match_rule(sentence, rule, defs)
        if result:
            return result
    return match_literal(sentence, negations, defs) #tipo verifica si en la oracion coincide con alguna regla y sino lo toma como enunciado solo es decir P o ~P

def match_rule(sentence, rule, defs):
    "Regla de coincidencia, que devuelve la traducción lógica y el dict de definiciones si la coincidencia tiene éxito."
    output, patterns = rule
    for pat in patterns:
        match = re.match(pat, sentence, flags=re.I)
        if match:
            groups = match.groupdict()
            for P in sorted(groups): # Aplicar recursivamente reglas a cada uno de los grupos coincidentes
                groups[P] = match_rules(groups[P], rules, defs)[0]
            return '(' + output.format(**groups) + ')', defs

def match_literal(sentence, negations, defs):
    "Ninguna regla coincide; la frase es un átomo. Añadir nueva proposición a defs. Manejar la negación."
    polarity = ''
    for (neg, pos) in negations:
        (sentence, n) = re.subn(neg, pos, sentence, flags=re.I)
        polarity += n * '～'
    sentence = clean(sentence)
    P = proposition_name(sentence, defs)
    defs[P] = sentence
    return polarity + P, defs

def proposition_name(sentence, defs, names='PQRSTUVWXYZBCDEFGHJKLMN'):
    "Devuelve el nombre antiguo de esta sentencia, si se usaba antes, o un nombre nuevo sin usar."
    inverted = {defs[P]: P for P in defs}
    if sentence in inverted:
        return inverted[sentence]                      # Buscar nombre usado anteriormente
    else:
        return next(P for P in names if P not in defs) # Utilizar un nuevo nombre sin usar

def clean(text):
    "Eliminar los espacios en blanco redundantes; tratar el apóstrofe y la coma/punto final."
    return ' '.join(text.split()).replace("’", "'").rstrip('.').rstrip(',')

In [5]:
match_rules("El planeta morirá debido a que la humanidad es tonta",rules,{})

('(P ⇒ Q)', {'P': 'la humanidad es tonta', 'Q': 'El planeta morirá'})

In [6]:
match_rules("iremos al monte y al parque",rules,{})

('(P ⋀ Q)', {'P': 'iremos al monte', 'Q': 'al parque'})

In [7]:
match_rules("No iremos de viaje a acapulco",rules,{})

('～P', {'P': 'si iremos de viaje a acapulco'})

In [8]:
clean("holaaa    f")

'holaaa f'

Por ejemplo:

In [9]:
match_rule("If loving you is wrong, I don't want to be right",
           Rule('{P} ⇒ {Q}', 'if {P}, {Q}'),
           {})

('(P ⇒ ～Q)', {'P': 'loving you is wrong', 'Q': 'I do want to be right'})

Here are some more test sentences and a top-level function to handle them:

In [10]:
sentences = '''
Polkadots and Moonbeams.
If you liked it then you shoulda put a ring on it.
If you build it, he will come.
It don't mean a thing, if it ain't got that swing.
If loving you is wrong, I don't want to be right.
Should I stay or should I go.
I shouldn't go and I shouldn't not go.
If I fell in love with you,
  would you promise to be true
  and help me understand.
I could while away the hours
  conferrin' with the flowers,
  consulting with the rain
  and my head I'd be a scratchin'
  while my thoughts are busy hatchin'
  if I only had a brain.
There's a federal tax, and a state tax, and a city tax, and a street tax, and a sewer tax.
A ham sandwich is better than nothing
  and nothing is better than eternal happiness
  therefore a ham sandwich is better than eternal happiness.
If I were a carpenter
  and you were a lady,
  would you marry me anyway?
  and would you have my baby.
Either Danny didn't come to the party or Virgil didn't come to the party.
Either Wotan will triumph and Valhalla will be saved or else he won't and Alberic will have
  the final word.
Sieglinde will survive, and either her son will gain the Ring and Wotan’s plan
  will be fulfilled or else Valhalla will be destroyed.
Wotan will intervene and cause Siegmund's death unless either Fricka relents
  or Brunnhilde has her way.
Figaro and Susanna will wed provided that either Antonio or Figaro pays and Bartolo is satisfied
  or else Marcellina’s contract is voided and the Countess does not act rashly.
If the Kaiser neither prevents Bismarck from resigning nor supports the Liberals,
  then the military will be in control and either Moltke's plan will be executed
  or else the people will revolt and the Reich will not survive'''.split('.')

import textwrap

def logic(sentences, width=80):
    "Match the rules against each sentence in text, and print each result."
    for s in map(clean, sentences):
        logic, defs = match_rules(s, rules, {})
        print('\n' + textwrap.fill('English: ' + s +'.', width), '\n\nLogic:', logic)
        for P in sorted(defs):
            print('{}: {}'.format(P, defs[P]))

logic(sentences)


English: Polkadots and Moonbeams. 

Logic: (P ⋀ Q)
P: Polkadots
Q: Moonbeams

English: If you liked it then you shoulda put a ring on it. 

Logic: (P ⇒ Q)
P: you liked it
Q: you shoulda put a ring on it

English: If you build it, he will come. 

Logic: (P ⇒ Q)
P: you build it
Q: he will come

English: It don't mean a thing, if it ain't got that swing. 

Logic: (～P ⇒ ～Q)
P: it is got that swing
Q: It do mean a thing

English: If loving you is wrong, I don't want to be right. 

Logic: (P ⇒ ～Q)
P: loving you is wrong
Q: I do want to be right

English: Should I stay or should I go. 

Logic: (P ⋁ Q)
P: Should I stay
Q: should I go

English: I shouldn't go and I shouldn't not go. 

Logic: (～P ⋀ ～～P)
P: I should go

English: If I fell in love with you, would you promise to be true and help me
understand. 

Logic: (P ⇒ (Q ⋀ R))
P: I fell in love with you
Q: would you promise to be true
R: help me understand

English: I could while away the hours conferrin' with the flowers, consulting
with th

Se ve muy bien. Pero lejos de ser perfecto.  Aquí hay algunos errores:

* `Should I stay` *etc.*:<br>las preguntas no son enunciados proposicionales.

* `If I were a carpenter`:<br>no maneja la lógica modal.

* `nothing is better`:<br>no maneja cuantificadores.

* `Either Wotan will triumph and Valhalla will be saved or else he won't`:<br>obtiene `'he will'` como una de las proposiciones, pero mejor sería que se refiriera a `'Wotan will triumph'`.

* `Wotan will intervene and cause Siegmund's death`:<br>consigue `"cause Siegmund's death"` como propuesta, pero mejor sería `"Wotan will cause Siegmund's death"`.

* `Figaro and Susanna will wed`:<br>consigue `"Figaro"` and `"Susanna will wed"` as two separate propositions; this should really be one proposition.

* `"either Antonio or Figaro pays"`:<br>gets `"Antonio"` como una proposición, pero debería ser `"Antonio pays"`.

* `If the Kaiser neither prevents`:<br>utiliza las proposiciones un tanto falsas `PQ` and `PR`. Esto debería hacerse de forma más limpia. El problema es el mismo que el problema anterior con Antonio: No tengo una buena manera de unir el sujeto de una frase verbal a las múltiples partes del verbo/objeto, cuando hay múltiples partes.



De seguro de que más frases de prueba revelarían muchos más tipos de errores.

También existe [una versión](proplogic.py) de este programa que está en Python 2 y utiliza sólo caracteres ASCII; si tiene un sistema Mac o Linux puede descargarlo como [`proplogic.py`](proplogic.py) y ejecutarlo con el comando `python proplogic.py`. O puedes ejecutarlo [online](https://www.pythonanywhere.com/user/pnorvig/files/home/pnorvig/proplogic.py?edit).

# **Librería para comparar oraciones**

In [11]:
!pip install sentence-transformers > /dev/null 2>&1

In [12]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [63]:
def silogismo_disyuntivo(not_R, P_or_Q):
    P,Q = re.split('⋁',P_or_Q)
    P=P.strip()
    Q=Q.strip()
    R=not_R.strip()
    #print(P,Q,R)
    if (R.strip('~') == P and '~' in not_R) or (R == P.strip('~') and '~' in P)  :
        return Q
    elif (R.strip('~') == Q and '~' in not_R) or (R == Q.strip('~') and '~' in Q) :
        return P
    else:
        print("No se puede llegar a una conclusion")

In [14]:
#SilogismoDisyuntivo
O3 = "I will go tomorrow or I will go yesterday"
O4 = "I won't go tomorrow"

In [15]:
logica1, proposiciones = match_rules(O3,rules,{})
logica2, proposicion = match_rules(O4,rules,{})

logica1= logica1.replace("(","")
logica1=logica1.replace(")","")
logica1= logica1.strip()


respuesta = silogismo_disyuntivo(logica2,logica1)

if respuesta == "P":
  print(proposiciones["P"])
elif respuesta == "Q":
  print(proposiciones["Q"])


I will go yesterday


In [16]:
#ModusTollens
O1 = "I will go to play if there is sun"
O2 = "There is sun"

In [17]:
def ModusTollens(R,P_Q):
    P,Q = re.split('⇒', P_Q)
    P = P.strip()
    Q = Q.strip()
    R = R.strip()
    #print(P,Q,R)
    if R == P: #Aquí quiero que compruebe si P coincide con el antecedente
        return Q
    elif R == '~'+Q :
        return '~'+P
    elif '~' in Q and R == Q.strip('~'):
        if '~' in P:
            return P.strip('~')
        else:
            return '~' + P
    else:
        print("No se puede llegar a una conclusion")


ModusTollens("~Q","P ⇒ Q")
ModusTollens("P","P ⇒ Q")

'Q'

In [18]:
logica1, proposiciones = match_rules(O1,rules,{})
P = proposiciones['P']
Q = proposiciones['Q']

logica2, proposicion = match_rules(O2,rules,{})
R=proposicion['P']

logica1= logica1.replace("(","")
logica1=logica1.replace(")","")
logica1= logica1.strip()

#comparamos similitud entre Q y R
embedding1 = model.encode(Q)
embedding2 = model.encode(R)
similarity = util.cos_sim(embedding1, embedding2)

if similarity>0.8:
  logica2="～Q"

respuesta=ModusTollens(logica2,logica1)

if respuesta=="P":
  print(proposiciones["P"])
elif respuesta=="~P":
  print("Se niega que "+proposiciones["P"])
elif respuesta=="Q":
  print(proposiciones["Q"])

I will go to play


In [19]:
def reglas_inferencia_dos_premisas():



SyntaxError: incomplete input (<ipython-input-19-2da9a76c4746>, line 2)

In [20]:
#DilemaConstructivo
O5 = "saldré siempre que hay sol"
O6 = "iré a plaza siempre que tengo dinero"
O7 = "hay sol o tengo dinero"

In [116]:
def DilemaConstructivo(PoQ,E_R,F_T):
    P,Q= re.split('⋁',PoQ)
    E,R = re.split('⇒',E_R)
    F,T = re.split('⇒',F_T)

    #print(P,Q,E,R,F,T)

    if (P.strip() == E.strip() and Q.strip() == F.strip()) or (P.strip() == F.strip() and Q.strip() == E.strip()):
        return R.strip() + ' ⋁ ' + T.strip()
    else:
        return "No se puede llegar a una conclusion"


In [82]:
DilemaConstructivo("P ⋁ R", "P ⇒ Q", "R ⇒ S")

'Q ⋁ S'

In [23]:
logica1, proposiciones1 = match_rules(O5,rules,{})

logica1= logica1.replace("(","")
logica1=logica1.replace(")","")
logica1= logica1.strip()

P = proposiciones1['P']
Q = proposiciones1['Q']

logica2, proposiciones2 =  match_rules(O6,rules,{})

logica2= logica2.replace("(","")
logica2=logica2.replace(")","")
logica2= logica2.strip()
logica2=logica2.replace("P","R")
logica2=logica2.replace("Q","S")

R = proposiciones2['P']
S = proposiciones2['Q']

logica3, proposiciones3 =  match_rules(O7,rules,{})

logica3= logica3.replace("(","")
logica3=logica3.replace(")","")
logica3= logica3.strip()

T = proposiciones3['P']
U = proposiciones3['Q']


embedding1 = model.encode(P)
embedding2 = model.encode(T)
similarity = util.cos_sim(embedding1, embedding2)

if similarity>0.9:
  logica3=logica3.replace("P","P")
  logica3=logica3.replace("Q","R")
else:
  logica3=logica3.replace("P","R")
  logica3=logica3.replace("Q","P")

respuesta=DilemaConstructivo(logica3,logica1,logica2)
if respuesta == "Q ⋁ S":
  print(proposiciones1['Q']+" o "+proposiciones2['Q'])

saldré o iré a plaza


In [24]:
#Simplificación
def simplificacion(P_and_Q):

    P,Q = re.split('⋀',P_and_Q)
    P = P.strip()
    Q = Q.strip()

    return P,Q

In [ ]:
O8="Dogs and cats"

In [25]:
logica1, proposiciones1 = match_rules(O8,rules,{})
logica1= logica1.replace("(","")
logica1=logica1.replace(")","")
logica1= logica1.strip()
P = proposiciones1['P']
Q = proposiciones1['Q']

resultado=simplificacion(logica1)
if resultado == ('P', 'Q'):
  print("- "+proposiciones1["P"])
  print("- "+proposiciones1["Q"])



NameError: name 'O8' is not defined

In [30]:
def ModusTollensAndPonens(R,P_Q):
    P,Q = re.split('⇒', P_Q)
    P = P.strip()
    Q = Q.strip()
    R = R.strip()
    #print(P,Q,R)
    if R == P: #Aquí quiero que compruebe si P coincide con el antecedente
        return Q
    elif R == '~'+Q :
        return '~'+P
    elif '~' in Q and R == Q.strip('~'):
        if '~' in P:
            return P.strip('~')
        else:
            return '~' + P
    else:
        print("No se puede llegar a una conclusion")

In [31]:
def SilogismoHipotetico(R_P,T_Q):
    T,Q = re.split('⇒', T_Q)
    R,P = re.split('⇒', R_P)#[0] Para que no me devuelva una lista como tal sino solo la primera parte de la cadena
    P = P.strip()
    Q = Q.strip()
    R = R.strip()
    T = T.strip()

    #print(P,Q,R)
    if P == T: #Aquí quiero que compruebe si P coincide con el antecedente
        return R + ' ⇒ ' + Q
    else:
        print("No se puede llegar a una conclusion")

In [68]:
def CaracteresComunes(Cadena1, Cadena2, ignorar):
    Set1 = set(Cadena1) # convierto las cadenas en un conjunto de caracteres
    Set2 = set(Cadena2)

    Set1 = Set1 - set(ignorar) # Le quitas los caracteres no deseados
    Set2 = Set2 - set(ignorar)

    # Comun = Set1 & Set2 # Se extraen los caracteres que coinciden de ambas cadenas

    print(Set1,Set2)

    if '⇒' in Set1 & Set2:
        print("Se ejecuta caso 1")
        return SilogismoHipotetico(Cadena1, Cadena2)
    elif '⇒' in Set1 or '⇒' in Set2:
        print("Se ejecuta caso 1.5")
        if '⇒' in Set1:
            resultado = ModusTollensAndPonens(Cadena2, Cadena1)
            print("Se ejecuta caso 2")
        elif '⇒' in Set2:
            resultado = ModusTollensAndPonens(Cadena1, Cadena2)
            print("Se ejecuta caso 3")

        if resultado != "No se puede llegar a una conclusión":
            return resultado
    else:
      print("el programa no tiene que ver con ->")

    if '⋁' in Set1 or '⋁' in Set2:
        print("Se ejecuta caso 3.5")
        if '⋁' in Set1:
            print("Se ejecuta caso 4")
            resultado = silogismo_disyuntivo(Cadena2, Cadena1)
        elif '⋁' in Set2:
            resultado = silogismo_disyuntivo(Cadena1, Cadena2)
            print("Se ejecuta caso 5")

        if resultado != "No se puede llegar a una conclusión":
            return resultado

ignorar = "()"
Propositions = ["~P ⋁ Q ", "P"]
#print(Propositions[0],Propositions[1])

~P ⋁ Q  P


In [118]:
def CaracteresComunes(Cadena1, Cadena2, Cadena3, ignorar):
    Set1 = set(Cadena1) #convierto las cadenas en un conjunto de caracteres
    Set2 = set(Cadena2)
    Set3 = set(Cadena3)

    Set1 = Set1 - set(ignorar) #Le quitas los caracteres no deseados
    Set2 = Set2 - set(ignorar)
    Set3 = Set3 - set(ignorar)
    #Comun = Set1 & Set2 #Se extraen los caracteres que coinciden de ambas cadenas

    if '⇒' in Set1:
        if '⇒' in Set2:
            print("Caso1")
            resultado = DilemaConstructivo(Cadena3, Cadena2,Cadena1)
        elif '⇒' in Set3:
            print("caso2")
            resultado = DilemaConstructivo(Cadena2, Cadena1,Cadena3)
            #print(resultado)

        if resultado != "No se puede llegar a una conclusion":
            return resultado
    elif '⇒' in Set2:
        if '⇒' in Set3:
            print("Caso3")
            resultado = DilemaConstructivo(Cadena1, Cadena2,Cadena3)
            #print(resultado)

        if resultado != "No se puede llegar a una conclusion":
            return resultado

    return "No se puede llegar a una conclusion"

In [119]:
Propositions = ["P ⇒ Q", "R ⇒ S","P ⋁ R"]
print(Propositions[0], Propositions[1], Propositions[2])
CaracteresComunes(Propositions[0],Propositions[1],Propositions[2],ignorar)

P ⇒ Q R ⇒ S P ⋁ R
Caso1


'S ⋁ Q'